## Train ResNets

### created by Yuying Liu, 04/30/2020

This script is a template for training neural network time-steppers for different systems and different time scales. To reproduce the results in the paper, one needs to obtain all 11 neural network models for each nonlinear system under study. For setup details, please refer to Table 2 in the paper.

In [1]:
import os
import sys
import torch
import numpy as np

import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import ResNet as net

ModuleNotFoundError: No module named 'ResNet'

In [ ]:
# adjustables

t = np.linspace(0, 8*np.pi, 500)
k = 0                         # model index: should be in {0, 2, ..., 10}
dt = t[1] - t[0]#0.01# 0.0005                     # time unit: 0.0005 for Lorenz and 0.01 for others
system = 'toy1_4x4'         # system name: 'Hyperbolic', 'Cubic', 'VanDerPol', 'Hopf' or 'Lorenz'
total_dim = 16
noise = 0.0                   # noise percentage: 0.00, 0.01 or 0.02

lr = 1e-3                     # learning rate
max_epoch = 100000            # the maximum training epoch 
batch_size = 320              # training batch size
arch = [total_dim, 128, 128, 128, total_dim]  # architecture of the neural network

In [ ]:
# paths
data_dir = os.path.join('../../data/', system)
model_dir = os.path.join('../../models/', system)

# global const
n_forward = 5
step_size = 2**k

In [ ]:
# load data
data = np.load(os.path.join(data_dir, "train_data.npy"))#inputs_2x2.npy"))#'train_noise{}.npy'.format(noise)))

# print(np.load("../../data/Hyperbolic/train_noise0.0.npy").shape)
# data = np.expand_dims(data, 1)
train_data = data[:75]
val_data = data[75:90]
test_data = data[90:]
# val_data = np.load(os.path.join(data_dir, 'val_noise{}.npy'.format(noise)))
# test_data = np.load(os.path.join(data_dir, 'test_noise{}.npy'.format(noise)))
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)
n_train = train_data.shape[0]
n_val = val_data.shape[0]
n_test = test_data.shape[0]
print(n_train)
print(n_val)
print(n_test)

print(train_data[0,:,:])

print("step_size*n_forward+1 = ", step_size*n_forward+1)
print("step_size*n_forward+1 = ", step_size*n_forward+1)
# create dataset object
dataset = net.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)

In [5]:
# for i in range(100):
#     plt.plot(train_data[i,:,0], train_data[i,:,1], 'b')

In [6]:
for k in [ 0, 1, 2, 3, 4, 5]:
    print("k = ", k)
    step_size = 2**k
    dataset = net.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)
    print(dataset.train_x.shape)
    model_name = 'model_D{}_noise{}_0.pt'.format(step_size, noise)

    # create/load model object
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model = torch.load(os.path.join(model_dir, model_name), map_location=device)
        model.device = device
    except:
        print('create model {} ...'.format(model_name))
        model = net.ResNet(arch=arch, dt=dt, step_size=step_size)

    # training
    model.train_net(dataset, max_epoch=30000, batch_size=batch_size, lr=lr,
                    model_path=os.path.join(model_dir, model_name))

k =  0
torch.Size([75, 16])
create model model_D1_noise0.0_0.pt ...
self.n_dim=  16
dataset.n_dim =  16
epoch 1000, training loss 0.000800661975517869, validation loss 0.0007432546699419618
(--> new model saved @ epoch 1000)
epoch 2000, training loss 0.00018583095516078174, validation loss 0.00033773586619645357
(--> new model saved @ epoch 2000)
epoch 3000, training loss 0.00012998783495277166, validation loss 0.00017593750089872628
(--> new model saved @ epoch 3000)
epoch 4000, training loss 0.00035132825723849237, validation loss 0.00024461597786284983
epoch 5000, training loss 4.483808152144775e-05, validation loss 7.184315472841263e-05
(--> new model saved @ epoch 5000)
epoch 6000, training loss 1.1908145097550005e-05, validation loss 2.5945400921045803e-05
(--> new model saved @ epoch 6000)
epoch 7000, training loss 9.780479740584269e-05, validation loss 7.898172771092504e-05
epoch 8000, training loss 6.109788228059188e-05, validation loss 9.144292562268674e-05
epoch 9000, traini

epoch 22000, training loss 0.0065950010903179646, validation loss 0.008466522209346294
epoch 23000, training loss 0.0017582603031769395, validation loss 0.008031478151679039
epoch 24000, training loss 0.0016350894002243876, validation loss 0.0036423071287572384
epoch 25000, training loss 0.0016973318997770548, validation loss 0.0039025302976369858
epoch 26000, training loss 0.003966671414673328, validation loss 0.005320501979440451
epoch 27000, training loss 0.0011922387639060616, validation loss 0.004126192070543766
epoch 28000, training loss 0.001018674112856388, validation loss 0.0042309933342039585
epoch 29000, training loss 0.0008127913461066782, validation loss 0.004091760143637657
epoch 30000, training loss 0.0007453003199771047, validation loss 0.0051584928296506405
k =  3
torch.Size([75, 16])
create model model_D8_noise0.0_0.pt ...
self.n_dim=  16
dataset.n_dim =  16
epoch 1000, training loss 0.019714761525392532, validation loss 0.02003205381333828
(--> new model saved @ epoc

epoch 19000, training loss 0.011006941087543964, validation loss 0.04462582990527153
epoch 20000, training loss 0.03538866341114044, validation loss 0.051801398396492004
epoch 21000, training loss 0.014074042439460754, validation loss 0.03495253622531891
epoch 22000, training loss 0.021338414400815964, validation loss 0.04007186368107796
epoch 23000, training loss 0.01744937151670456, validation loss 0.04613374546170235
epoch 24000, training loss 0.0397237204015255, validation loss 0.04821658134460449
epoch 25000, training loss 0.029618006199598312, validation loss 0.052468977868556976
epoch 26000, training loss 0.04327574744820595, validation loss 0.06654207408428192
epoch 27000, training loss 0.054270509630441666, validation loss 0.069190613925457
epoch 28000, training loss 0.030530575662851334, validation loss 0.042260754853487015
epoch 29000, training loss 0.03779932111501694, validation loss 0.0634644627571106
epoch 30000, training loss 0.04398011416196823, validation loss 0.07071

In [7]:
 

.

SyntaxError: invalid syntax (<ipython-input-7-a5d5b61aa8a6>, line 1)